In [4]:
from modules import *
from functions import *

In [11]:
""" Track final weights """

x = np.linspace(0, 0.3, 100)

N = int(1000)                          # number of neurons 
theta_hat = 90                        # angle to which output neuron is tuned
theta_stim = 90                       # stimulus angle
all_angles = np.linspace(0, 180, N)   # stimulus space
n_test_angles = 500

def tweak_params(a, b, rand_scaling=1, hebb_scaling=1, learning_rate=0.01, n_thetas=100, n_trials=50):

    W = circular_gaussian(N, theta_hat, amp=1, sigma=15, baseline=0); W /= np.sum(W)
    POs = []; ratios = []
    for trial in range(n_trials):

        H = hebbian_component(N, W, n_thetas, theta_stim, type='baseline')
        eta = abs(np.random.randn(N))
        hebb_component = hebb_scaling * H * propensity_hebb(W, a)
        rand_component = rand_scaling * eta * propensity_rand(W, b)   #  both use a for the moment
        W += (hebb_component + rand_component) * learning_rate
        normalisation(W) 
        # print(hebb_component.mean(), rand_component.mean())
        ratios.append(np.mean(hebb_component) / (np.mean(rand_component) + 10e-10))
        PO = get_preferred_orientation(N, W, n_angles=n_test_angles)
        POs.append(PO)

    print('Median ratio: ', np.median(ratios))

    fig, axs = plt.subplots(1, 4, figsize=(14, 4), dpi=150)
    markerline, stemline, baseline = axs[0].stem(all_angles, W, basefmt='k')
    plt.setp(stemline, linewidth = 1, color='silver') 
    plt.setp(markerline, markersize = 3, color='k')

    axs[0].axvline(theta_hat, color='r', linestyle='--', linewidth=1.5) 
    axs[0].axvline(theta_stim, color='orange', linestyle='--', linewidth=1.5) 
    # axs[0].axvline(np.sum((np.linspace(0, 180, N) * W)), color='b', linestyle='--', linewidth=1.5)    # mean of distribution 
    axs[0].set_xticks([0, 90, 180])
    axs[0].set_xlabel(r'$\mathbf{\theta} \; [^\circ] $', fontweight='bold')
    axs[0].set_ylabel(r'Weight', fontweight='bold')
    axs[0].locator_params(nbins=3)
    axs[0].set_ylim(0, 0.05)

    axs[1].plot(POs, c='k')
    # duplicate y axis
    ax2 = axs[1].twinx()
    ax2.plot(ratios, c='r', alpha=0.5)
    ax2.set_ylim(0.5, 1.5)
    axs[1].axhline(theta_hat, color='r', linestyle='--', linewidth=1.5)
    axs[1].set_ylim(theta_hat - 20, theta_hat + 20)
    # axs[1].set_ylim(10, 90)
    axs[1].set_xlabel(r'Trials', fontweight='bold')
    axs[1].set_ylabel(r'PO $[^\circ]$', fontweight='bold')
    axs[1].locator_params(nbins=4)

    axs[2].plot(x, propensity_hebb(x, a), c='firebrick', lw=1.5)
    axs[2].set_xlabel(r'Weight', fontweight='bold')
    axs[2].set_ylabel(r'Propensity', fontweight='bold')
    axs[2].set_ylim(0, 1.01)
    axs[2].set_xlim(0, 0.3)
    axs[2].locator_params(axis='y', nbins=3)

    axs[3].plot(x, propensity_rand(x, b), c='firebrick', lw=1.5)
    axs[3].set_xlabel(r'Weight', fontweight='bold')
    axs[3].set_ylabel(r'Rand propensity', fontweight='bold')
    axs[3].set_ylim(0, 1.01)
    axs[3].set_xlim(0, 0.3)
    axs[3].locator_params(axis='y', nbins=3)
    fig.tight_layout(pad=2.5)

widget = interactive(tweak_params, a=30, b=30, rand_scaling=(0, 2, 0.1), hebb_scaling=(0, 1, 0.1), learning_rate=(0, 0.1, 0.001), n_thetas=(1, 180), n_trials=(1, 200))
controls = HBox(widget.children[:-1], layout=Layout(flex_flow='row wrap'))
output = widget.children[-1]
# display(widget) 
display(VBox([controls, output]))

In [48]:
""" Track weights over trials """

N = int(500)                          # number of neurons 
theta_hat = 20                        # angle to which output neuron is tuned 
theta_stim = 90                       # stimulus angle
all_angles = np.linspace(0, 180, N)   # stimulus space
n_trials = 100 
n_thetas = 100
a=10    
b=10 

learning_rate = 0.1
hebb_scaling = 1
rand_scaling = 0

W_init = circular_gaussian(N, theta_hat, amp=1, sigma=15, baseline=0); W_init /= np.sum(W_init)
W = np.zeros((N, n_trials+1)); W[:, 0] = W_init
hebbian = np.zeros((N, n_thetas, n_trials)); sum_hebbian = np.zeros((N, n_trials));
prop_by_sum = np.zeros((N, n_trials)); prop_by_rand = np.zeros((N, n_trials)); total = np.zeros((N, n_trials))

POs = []
for trial in range(n_trials):
    
    thetas = np.random.choice(180, size=n_thetas, replace=False) 
    hebbs = np.zeros((N, n_thetas))
    for i in range(n_thetas):
        theta = theta_stim 
        # theta = thetas[i]
        u = circular_gaussian(N, theta, amp=0.106, sigma=60, baseline=0)
        v = W[:, trial].T.dot(u)  
        hebbs[:, i] = (u * v)
        
    random = abs(np.random.randn(N)) * rand_scaling 
    sum_hebb = np.sum(np.array(hebbs), axis=1) * hebb_scaling 
    prop_hebb = propensity_hebb(W[:, trial], a)
    prop_rand = propensity_rand(W[:, trial], b)

    hebbian[:, :, trial] = hebbs; sum_hebbian[:, trial] = sum_hebb; prop_by_sum[:, trial] = prop_hebb * sum_hebb ; prop_by_rand[:, trial] = prop_rand * random
    total[:, trial] = (prop_hebb * sum_hebb + prop_rand * random)

    W_new = W[:, trial] + (prop_hebb * sum_hebb + prop_rand * random) * learning_rate
    normalisation(W_new); W[:, trial+1] = W_new

    PO = get_preferred_orientation(N, W[:, trial], n_angles=500)
    POs.append(PO)

In [49]:
def plot_weights(trial=n_trials-1):
    fig, axs = plt.subplots(1, 3, figsize=(12, 3.5), dpi=180)

    offset = 3
    lim = -3
    scale = 1.2
    for i in range(n_thetas):
        axs[0].plot(hebbian[:, i, trial] / scale + offset, c='k', alpha=0.2)

    axs[0].plot(sum_hebbian[:, trial], c='lightskyblue') #, clip_on=False)
    axs[0].plot(prop_by_sum[:, trial], lw=1.2, c='yellowgreen')
    axs[0].plot(prop_by_rand[:, trial], lw=1.2, c='darkorange')
    
    axs[0].plot(total[:, trial], lw=0.7, c='k')
    axs[0].axis('off')  
    axs[0].plot([0, N], [lim, lim], c='k', lw=2)
    axs[0].set_ylim(lim, 5)

    markerline, stemline, baseline = axs[1].stem(all_angles, W[:, trial]*0.9, basefmt='k')
    plt.setp(stemline, linewidth = 1, color='silver') #, clip_on=False)
    plt.setp(markerline, markersize = 3, color='k') #, clip_on=False)
    axs[1].axvline(theta_stim, color='orange', linestyle='--', linewidth=1.5)
    axs[1].axvline(theta_hat, color='r', linestyle='--', linewidth=1.5) 
    axs[1].set_xticks([0, 90, 180])
    axs[1].set_xlabel(r'$\mathbf{\theta} \; [^\circ] $', fontweight='bold')
    axs[1].set_ylabel(r'Weight', fontweight='bold')
    axs[1].locator_params(nbins=3)
    # axs[1].set_ylim(-0.001, 0.03)
    axs[1].set_yticks([0, 0.04])

    axs[2].plot(POs[:trial], c='k')
    axs[2].axhline(theta_hat, color='r', linestyle='--', linewidth=1.5)
    # axs[2].set_ylim(60, 120)
    axs[2].set_ylim(10, 90)
    axs[2].set_xlabel(r'Trials', fontweight='bold')
    axs[2].set_ylabel(r'PO $[^\circ]$', fontweight='bold')
    axs[2].locator_params('y', nbins=4)
    # axs[2].set_yticks([70, 90, 110])

    fig.subplots_adjust(wspace=0.5)
    sns.despine()
    
interact(plot_weights, trial=(0, n_trials));

interactive(children=(IntSlider(value=99, description='trial'), Output()), _dom_classes=('widget-interact',))